In [1]:
# Written by Abhijeet Kulkarni @amkulk@udel.edu
import sympy as sm
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import dill
dill.settings['recurse'] = True
from sympy.utilities.lambdify import lambdify, implemented_function
!which python

/media/aamanku/Dataa/UserFiles/Documents/GitHub/AutonomusDriving/forautonomous/bin/python


In [2]:
M,I,Fx1,Fy1,Fx2,Fy2,a1,a2,C1,C2 = sm.symbols(r'M I F_{x1} F_{y1} F_{x2} F_{y2} a_1 a_2 C_1 C_2',real=True)
vx,vy,dtheta,theta,delta = sm.symbols(r'v_x v_y \dot{\theta} \theta \delta',real=True)
X,Y,dX,dY = sm.symbols(r'X Y \dot{X} \dot{Y}')

In [3]:
#Velocities
vx = dX*sm.cos(theta) + dY*sm.sin(theta)
vy = -dX*sm.sin(theta) + dY*sm.cos(theta)
#Forces
#Fy1 = -C1*sm.atan((vy+dtheta*a1)/vx)
#Fy2 = -C2*sm.atan((vy-dtheta*a2)/vx)
Fy = Fy2 + Fy1*sm.cos(delta)+Fx1*sm.sin(delta)
Fx = Fx2 + Fx1*sm.cos(delta) - Fy1*sm.sin(delta)
Tau = -Fy2*a2+a1*(Fy1*sm.cos(delta)+Fx1*sm.sin(delta))

#Accelerations in Global frame
ddX = (Fx*sm.cos(theta)-Fy*sm.sin(theta))/M
ddY = (Fx*sm.sin(theta)+Fy*sm.cos(theta))/M
ddtheta = Tau/I
ddX=sm.simplify(ddX)
ddY=sm.simplify(ddY)
ddtheta=sm.simplify(ddtheta)
ode=sm.simplify(sm.Matrix([dX, dY, dtheta, ddX, ddY, ddtheta]))
fode=lambdify([[X,Y,theta,dX,dY,dtheta],[delta,Fx1,Fx2],[Fy1,Fy2],M,I,a1,a2],ode,'numpy')
ode
dill.dump(fode, open("eta_dot.dill","wb"))

Matrix([
[                                                                                                \dot{X}],
[                                                                                                \dot{Y}],
[                                                                                           \dot{\theta}],
[(F_{x1}*cos(\delta + \theta) + F_{x2}*cos(\theta) - F_{y1}*sin(\delta + \theta) - F_{y2}*sin(\theta))/M],
[(F_{x1}*sin(\delta + \theta) + F_{x2}*sin(\theta) + F_{y1}*cos(\delta + \theta) + F_{y2}*cos(\theta))/M],
[                                        (-F_{y2}*a_2 + a_1*(F_{x1}*sin(\delta) + F_{y1}*cos(\delta)))/I]])

In [4]:
#ddX.free_symbols
#ddY.free_symbols
#ddtheta.free_symbols

In [12]:
#Contact forces
Fy1 = -C1*(sm.atan2((vy+dtheta*a1),vx)-delta)
Fy2 = -C2*(sm.atan2((vy-dtheta*a2),vx))
Fy1
Fy2
#fF_contact = lambdify([[x,y,bet,dx,dy,dbet],[tht,Ffl,Frl],M,I,a,b],F_contact,'numpy')
F_lateral = sm.Matrix([Fy1,Fy2])
fF_lateral = lambdify([[X,Y,theta,dX,dY,dtheta],[delta,Fx1,Fx2],a1,a2,C1,C2],F_lateral,'numpy')
fF_lateral([1,1,np.deg2rad(180),-1,0,0],[0,0,0],1,1,1000,1000)
#dill.dump(fF_lateral, open("F_contact.dill","wb"))

-C_1*(-\delta + atan2(-\dot{X}*sin(\theta) + \dot{Y}*cos(\theta) + \dot{\theta}*a_1, \dot{X}*cos(\theta) + \dot{Y}*sin(\theta)))

-C_2*atan2(-\dot{X}*sin(\theta) + \dot{Y}*cos(\theta) - \dot{\theta}*a_2, \dot{X}*cos(\theta) + \dot{Y}*sin(\theta))

array([[-1.2246468e-13],
       [-1.2246468e-13]])

<function _lambdifygenerated(_Dummy_185, _Dummy_186, a_1, a_2, C_1, C_2)>